In [41]:
import json
import math
import numpy as np
import pandas as pd
import geopandas as gpd

In [42]:
def hazard_ratio(theta, alpha, miu, v, concentration):
    z = (concentration-2.4) if (concentration - 2.4) > 0 else 0
    numerator = theta * np.log(z/(alpha)+1)
    denominator = 1 + np.exp(-(z-miu)/v)
    GEMM = np.exp(numerator/denominator)
    return GEMM 
def hazard_ratio_NCDLRI(concentration):
    if concentration < 2.4: # Lowest observed concentration in any of the 41 cohorts
        return 1
    else:
        return hazard_ratio(0.1430, 1.6, 15.5, 36.8, concentration)

In [43]:
# fraction = pd.read_csv('comparison/country_classify.csv')
fraction = pd.read_csv('C:/Users/15311/Desktop/country_classify.csv')

In [44]:
# fraction = fraction[195:205]

In [45]:
with open("NCDLRI_Total.json",'r') as load_f: # ../2_Concentration_Factors/
    NCDLRI_Total = json.load(load_f)
Country = list(NCDLRI_Total.keys())

### Add the total wholeworld concentration together

In [1]:
# For ROW
A = ['agr','ene','ind','nrtr','rcor','rcoc','rcoo','shp','slv','wst']
for j in A:
    read_shp = gpd.read_file(r'output/output_wholeworld/wholeworld_' + j + '.' + 'shp')
    fraction['TotalPM25'] += read_shp['TotalPM25']

output/output_wholeworld/wholeworld_agr.shp
output/output_wholeworld/wholeworld_ene.shp
output/output_wholeworld/wholeworld_ind.shp
output/output_wholeworld/wholeworld_nrtr.shp
output/output_wholeworld/wholeworld_rcor.shp
output/output_wholeworld/wholeworld_rcoc.shp
output/output_wholeworld/wholeworld_rcoo.shp
output/output_wholeworld/wholeworld_shp.shp
output/output_wholeworld/wholeworld_slv.shp
output/output_wholeworld/wholeworld_wst.shp


## Baseline incidence rate
**I**: Reported incidence rate
<br>
**p**: population within a grid cell
<br>
**z**: concentration within a grid cell
<br>
**hr** specifies the hazard ratio as a function of **z**
<br>
**
for country in all_country:
<br>
$\;\;\;\;$ for i in all_gridcells:
<br>
$\;\;\;\;$ $\;\;\;\;$ Hsum += p[i] * hr(z[i])
<br>
$\;\;\;\;$ $\;\;\;\;$ Psum += p[i]
<br>
$\;\;\;\;$ Hbar = Hsum/Psum
<br>
**
### Question: if a grid cell is made up of 1/4 USA and 3/4 CAN, which concentration (z[i]) is correct to calculate Hsum += p[i] * hr(z[i]):
<br>
### 1. For USA and CAN, both use z[i] √
<br>
### 2. For USA, use 1/4 * z[i]; For CAN, use 3/4 * z[i]

In [38]:
# Country = ['ROW','USA']

In [39]:
baseline = {}
Total_pop = {}
for country in Country:
    Hsum = 0 # hazard
    Psum = 0 # population
    for index in range(len(fraction)):
#         print('population',fraction.iloc[index]['TotalPop'])
        if country not in json.loads(fraction.iloc[index]['area_fraction'].replace('\'','\"')).keys():
            continue
        frac = json.loads(fraction.iloc[index]['area_fraction'].replace('\'','\"'))[country]
        Hsum += frac * fraction.iloc[index]['TotalPop'] * hazard_ratio_NCDLRI(fraction.iloc[index]['TotalPM25']) 
        Psum += frac * fraction.iloc[index]['TotalPop']
#     print('Psum',Psum)
    Hbar = Hsum/Psum
    Total_pop[country] = Psum
    baseline[country] = NCDLRI_Total[country]/Hbar

In [40]:
# baseline

In [ ]:
file=open("comparison/Total_pop.json",'w')
json.dump(Total_pop,file)
file.close()

In [ ]:
file=open("comparison/Baseline.json",'w')
json.dump(baseline,file)
file.close()